# Nettoyage de tweets pour LLM

L'objectif de ce notebook est de préparer un corpus de tweets pour un usage avec un modèle de langage (LLM).

On va :

1. Charger un fichier CSV d'export Twitter.
2. Harmoniser les noms de colonnes (id, texte, etc.).
3. Séparer les tweets :
   - des **comptes Free** (annonces, réponses, retweets de Free),
   - des **clients** (utilisateurs finaux).
4. Nettoyer le texte des tweets clients :
   - suppression des URLs ;
   - normalisation des sauts de ligne (`\n`, `\\n`, `\r`, `\t`) en espaces ;
   - conservation des hashtags ;
   - passage en minuscules.
5. Dédupliquer les tweets sur l’identifiant `id`.
6. Produire des statistiques de contrôle (nombre de tweets, retweets Free, hashtags perdus…).

À la fin, on obtient un fichier CSV propre contenant uniquement des tweets clients nettoyés, prêt pour le LLM.


In [9]:
import re
import html
import pandas as pd
from pathlib import Path

## 1. Fonctions utilitaires (regex, normalisation, hashtags, noms de colonnes)

Dans cette partie, on définit :

- des **expressions régulières** pour détecter :
  - les URLs ;
  - les retweets (tweets commençant par `RT`).
- une fonction `normalize_whitespace_tokens` qui transforme toutes les variantes de sauts de ligne et tabulations (`\n`, `\\n`, `\r`, `\\r`, `\t`, `\\t`) en espaces ;
- une fonction `extract_hashtags` qui extrait la liste des hashtags à partir d'un texte **déjà normalisé** ;
- une fonction `_normalize_col_name` qui homogénéise les noms de colonnes (minuscules, underscores, suppression de caractères cachés).

In [10]:
# Regex globales

URL_PAT = re.compile(r'https?://\S+|www\.\S+', flags=re.IGNORECASE)
RT_PAT  = re.compile(r'^\s*rt\b', flags=re.IGNORECASE)  # RT au début de texte

# Utils texte

def normalize_whitespace_tokens(s: str) -> str:
    """
    Normalise les sauts de ligne / tabulations :
    - remplace les séquences texte '\\n', '\\r', '\\t'
    - ET les vrais caractères '\n', '\r', '\t'
    par des espaces.
    """
    if not isinstance(s, str):
        return s

    # Séquences échappées (backslash + lettre)
    s = s.replace("\\r", " ").replace("\\n", " ").replace("\\t", " ")
    # Caractères réels de contrôle
    s = s.replace("\r", " ").replace("\n", " ").replace("\t", " ")
    return s


def extract_hashtags(s_norm: str):
    """
    Extrait les hashtags d'une chaîne DEJA normalisée
    (pas de \\n / \\t bizarres).
    Retourne une liste de tags commençant par '#'.
    """
    if not isinstance(s_norm, str):
        return []
    return re.findall(r'#[^\s#]+', s_norm, flags=re.UNICODE)


def _normalize_col_name(c) -> str:
    """
    Normalise les noms de colonnes :
    - supprime le BOM éventuel
    - strip
    - lower
    - remplace les espaces par des underscores
    """
    return (
        str(c)
        .replace('\ufeff', '')  # BOM éventuel
        .strip()
        .lower()
        .replace(" ", "_")
    )


## 2. Fonction de nettoyage du texte pour le LLM

La fonction `clean_llm_text` prend un texte **déjà normalisé** (sans `\n` bizarres) et :

1. Décode les entités HTML (`&amp;`, `&quot;`, etc.).
2. Supprime les URLs.
3. Normalise les espaces (un seul espace entre les mots).
4. Vérifie que les hashtags présents dans le texte normalisé initial n'ont pas été perdus
   et réinjecte ceux qui manquent à la fin du texte.
5. Passe tout en minuscules (`casefold`).

Le résultat est stocké dans la colonne `text_clean_llm`.


In [11]:
def clean_llm_text(text_norm: str) -> str:
    """
    Nettoie un texte de tweet DEJA normalisé (text_norm) pour usage LLM :
    1. Décodage HTML (&amp;, &quot;, etc.).
    2. Suppression des URLs.
    3. Normalisation des espaces.
    4. Vérification / réinjection des hashtags perdus.
    5. Passage en minuscules (casefold).
    """
    if not isinstance(text_norm, str):
        text_norm = str(text_norm) if text_norm is not None else ""

    # Décodage HTML (&amp; -> &, etc.)
    cleaned = html.unescape(text_norm)

    # Suppression des URLs
    cleaned = URL_PAT.sub(' ', cleaned)

    # Normalisation des espaces (un seul espace entre les mots)
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()

    # Gestion des hashtags possiblement perdus
    raw_tags     = extract_hashtags(text_norm)   # hashtags sur texte brut normalisé
    cleaned_tags = extract_hashtags(cleaned)     # hashtags sur texte nettoyé

    cleaned_lower = {t.casefold() for t in cleaned_tags}
    missing = [t for t in raw_tags if t.casefold() not in cleaned_lower]

    if missing:
        if cleaned and not cleaned.endswith(' '):
            cleaned += ' '
        cleaned += ' '.join(missing)
        cleaned = re.sub(r'\s+', ' ', cleaned).strip()

    # Minuscule robuste
    return cleaned.casefold()


## 3. Lecture robuste du fichier CSV

La fonction `read_csv_robust` permet de charger le fichier CSV d'export Twitter :

- essaie d'abord le format `UTF-8` ;
- en cas d'échec, réessaie en `UTF-8-SIG` (fréquent pour les exports Excel) ;
- laisse `pandas` détecter automatiquement le séparateur ;
- force toutes les colonnes en chaînes (`dtype=str`).

On obtient un `DataFrame` brut qui sera ensuite nettoyé.


In [12]:
def read_csv_robust(path: Path) -> pd.DataFrame:
    """
    Essaie UTF-8 puis UTF-8-SIG (Excel) avec détection automatique du séparateur.
    Tout est lu en string (dtype=str).
    """
    try:
        return pd.read_csv(
            path,
            sep=None,
            engine="python",
            encoding="utf-8",
            on_bad_lines="skip",
            dtype=str,
        )
    except Exception:
        return pd.read_csv(
            path,
            sep=None,
            engine="python",
            encoding="utf-8-sig",
            on_bad_lines="skip",
            dtype=str,
        )


## 4. Pipeline global de nettoyage des tweets

La fonction `clean_dataframe(df)` applique tout le pipeline :

1. Normalisation des noms de colonnes (minuscules, underscore, suppression BOM).
2. Vérification de la présence de la colonne `id` (utilisée pour dédupliquer).
3. Détection de la colonne texte (`full_text`, `text`, `tweet` ou `content`) et copie dans `text_raw`.
4. Normalisation de `text_raw` avec `normalize_whitespace_tokens` (tous les `\n`, `\\n`, `\r`, `\\r`, `\t`, `\\t` → espaces).
5. Détection des retweets sur tout le dataset (`is_retweet`) :
   - texte commençant par `RT` ;
   - ou `retweeted_status` non nul si la colonne existe.
6. Filtre des comptes Free (`free`, `free_1337`, `freebox`, `freefoot`, `freenewsactu`, `universfreebox`) :
   - comptage des tweets Free filtrés (`reste`) ;
   - comptage des retweets Free (`retweets_free`) ;
   - conservation uniquement des tweets **clients** dans `df`.
7. Extraction des hashtags bruts des tweets clients → colonne `hashtags_list`.
8. Nettoyage du texte client pour le LLM → colonne `text_clean_llm`.
9. Contrôle des hashtags perdus → colonne `lost_hashtags_count`.
10. Déduplication sur `id` (un tweet client unique par id).
11. Calcul des statistiques globales (`rows_in`, `rows_out`, `retweets_free`, `retweets_total`, `reste`, `dedup_removed`, `lost_hashtags_total`).

La fonction renvoie :
- `df` : DataFrame nettoyé (tweets clients),
- `stats` : dictionnaire de statistiques de nettoyage.


In [13]:
def clean_dataframe(df: pd.DataFrame):
    """
    Pipeline complet de nettoyage des tweets.
    """
    orig_len = len(df)

    # Normalisation des noms de colonnes (avec gestion BOM)
    df = df.copy()
    df.columns = [_normalize_col_name(c) for c in df.columns]

    # Vérification de la colonne 'id'
    if "id" not in df.columns:
        raise SystemExit(
            "La colonne 'id' est obligatoire pour dédupliquer les tweets.\n"
            f"Colonnes disponibles après normalisation : {', '.join(df.columns)}"
        )

    # Détection de la colonne texte
    text_col = next((c for c in ["full_text", "text", "tweet", "content"] if c in df.columns), None)
    if not text_col:
        raise SystemExit("Colonne texte introuvable (full_text / text / tweet / content).")

    # Texte brut -> text_raw
    df["text_raw"] = df[text_col].astype(str)

    # Normaliser UNE FOIS text_raw (il devient notre texte de référence "propre")
    df["text_raw"] = df["text_raw"].apply(normalize_whitespace_tokens)

    # Détection des retweets sur TOUT le dataset (avant filtre Free)
    has_retweeted_status = "retweeted_status" in df.columns
    if has_retweeted_status:
        retweeted_status_notna = df["retweeted_status"].notna()
    else:
        retweeted_status_notna = False

    df["is_retweet"] = df["text_raw"].str.match(RT_PAT, na=False) | retweeted_status_notna

    # Filtre des comptes Free (annonces / réponses / retweets de Free)
    POSSIBLE_HANDLE_COLS = [
        "screen_name", "user_screen_name", "username",
        "user", "author", "account", "handle"
    ]
    handle_col = next((c for c in POSSIBLE_HANDLE_COLS if c in df.columns), None)
    removed = 0
    retweets_free = 0  # retweets dans les lignes filtrées (Free)

    if handle_col:
        df["__handle_norm"] = (
            df[handle_col].astype(str)
            .str.replace(r"^@", "", regex=True)
            .str.strip()
            .str.casefold()
        )
        BLOCKED = {"free", "free_1337", "freebox", "freefoot", "freenewsactu", "universfreebox"}
        mask_free = df["__handle_norm"].isin(BLOCKED)

        removed = int(mask_free.sum())
        # Retweets parmi les tweets Free (avant de les enlever)
        retweets_free = int(df.loc[mask_free, "is_retweet"].sum())

        # On enlève les lignes Free pour ne garder que les tweets client
        df = df[~mask_free].drop(columns=["__handle_norm"])
    else:
        mask_free = None  # cohérence

    # Hashtags bruts sur les tweets clients (à partir de text_raw déjà normalisé)
    df["hashtags_list"] = df["text_raw"].apply(extract_hashtags)

    # Texte nettoyé LLM à partir de text_raw (déjà normalisé)
    df["text_clean_llm"] = df["text_raw"].apply(clean_llm_text)

    # Contrôle des hashtags perdus
    def _lost_hashtags_count_row(row):
        raw_list = row.get("hashtags_list")
        if not isinstance(raw_list, list):
            raw_list = []
        rset = {t.casefold() for t in raw_list}
        cset = {t.casefold() for t in extract_hashtags(row.get("text_clean_llm", ""))}
        return max(0, len(rset - cset))

    df["lost_hashtags_count"] = df.apply(_lost_hashtags_count_row, axis=1)

    # Déduplication uniquement sur id
    before = len(df)
    df["id"] = df["id"].astype(str).str.strip()
    df = df.dropna(subset=["id"]).drop_duplicates(subset=["id"], keep="first")
    dedup_removed = before - len(df)

    # Stats
    rows_out = int(len(df))
    retweets_client = int(df["is_retweet"].sum())  # côté clients (non affiché)
    retweets_total = retweets_client + retweets_free
    reste = int(removed)  # tweets Free filtrés

    stats = {
        "rows_in": int(orig_len),
        "rows_out": rows_out,
        "tweets_client": rows_out,
        "retweets_free": retweets_free,
        "retweets_total": retweets_total,
        "reste": reste,
        "dedup_removed": int(dedup_removed),
        "filtered_handles": int(removed),
        "lost_hashtags_total": int(df["lost_hashtags_count"].sum()),
    }
    return df, stats


## 5. Chargement des données brutes

On charge maintenant le fichier CSV d’export des tweets (par exemple `free_tweet_export.csv`)
et on affiche les premières lignes pour vérifier la structure (colonnes `id`, texte, handle, etc.).


In [14]:
input_path = Path("../data/free_tweet_export.csv")

if not input_path.exists():
    raise SystemExit(f"Fichier introuvable : {input_path.resolve()}")

df_raw = read_csv_robust(input_path)
df_raw.head()


,﻿id,created_at,full_text,media,screen_name,name,profile_image_url,user_id,in_reply_to,retweeted_status,...,favorite_count,retweet_count,bookmark_count,quote_count,reply_count,views_count,favorited,retweeted,bookmarked,url
0,1343458257915031553,2020-12-28 08:26:23 +01:00,"💩 à @free parce-que Débit Très instable, … \n\...",[],m_annuel,M Annuel,https://abs.twimg.com/sticky/default_profile_i...,1104790986801250304,NaN,NaN,...,2,1,0,0,1,NaN,false,false,false,https://twitter.com/m_annuel/status/1343458257...
1,1393158240083587075,2021-05-14 12:56:22 +02:00,"RT @free: Retrouvez désormais @ToonamiFR, la c...","[{""type"":""photo"",""url"":""https://t.co/kuAYafYDi...",Freebox,Assistance Freebox,https://pbs.twimg.com/profile_images/671676021...,58920430,NaN,1393125178591248387,...,0,16,0,0,0,NaN,false,false,false,https://twitter.com/Freebox/status/13931582400...
2,1403337211475546112,2021-06-11 15:03:58 +02:00,"RT @free: A suivre ce soir, le 1er match de l’...","[{""type"":""photo"",""url"":""https://t.co/gMTcYtGdd...",Freebox,Assistance Freebox,https://pbs.twimg.com/profile_images/671676021...,58920430,NaN,1403329474561724421,...,0,15,0,0,0,NaN,false,false,false,https://twitter.com/Freebox/status/14033372114...
3,1403337257571004417,2021-06-11 15:04:09 +02:00,RT @free: Disponible sur le canal 101 avec les...,[],Freebox,Assistance Freebox,https://pbs.twimg.com/profile_images/671676021...,58920430,NaN,1403332744558661636,...,0,5,0,0,0,NaN,false,false,false,https://twitter.com/Freebox/status/14033372575...
4,1418550491034882052,2021-07-23 14:36:07 +02:00,« Faites vos premiers pas avec nous ! Découvre...,"[{""type"":""video"",""url"":""https://t.co/YCMv79evb...",Freebox,Assistance Freebox,https://pbs.twimg.com/profile_images/671676021...,58920430,NaN,NaN,...,31,7,0,3,35,NaN,false,false,false,https://twitter.com/Freebox/status/14185504910...


## 6. Application du pipeline de nettoyage

On applique la fonction `clean_dataframe` au DataFrame brut.

On obtient :

- `df_clean` : les tweets **clients** nettoyés ;
- `stats` : un dictionnaire avec les principales statistiques (volume avant/après, tweets Free filtrés, retweets, hashtags perdus, etc.).


In [15]:
df_clean, stats = clean_dataframe(df_raw)

print("✅ Nettoyage terminé.")
for k, v in stats.items():
    print(f" - {k}: {v}")

df_clean.head()


✅ Nettoyage terminé.
 - rows_in: 6375
 - rows_out: 3044
 - tweets_client: 3044
 - retweets_free: 16
 - retweets_total: 16
 - reste: 3331
 - dedup_removed: 0
 - filtered_handles: 3331
 - lost_hashtags_total: 0


,id,created_at,full_text,media,screen_name,name,profile_image_url,user_id,in_reply_to,retweeted_status,...,views_count,favorited,retweeted,bookmarked,url,text_raw,is_retweet,hashtags_list,text_clean_llm,lost_hashtags_count
0,1343458257915031553,2020-12-28 08:26:23 +01:00,"💩 à @free parce-que Débit Très instable, … \n\...",[],m_annuel,M Annuel,https://abs.twimg.com/sticky/default_profile_i...,1104790986801250304,NaN,NaN,...,NaN,false,false,false,https://twitter.com/m_annuel/status/1343458257...,"💩 à @free parce-que Débit Très instable, … F...",False,"[#fing, #internet, #Free]","💩 à @free parce-que débit très instable, … fre...",0
20,1480465391424053249,2022-01-10 10:03:49 +01:00,🎂 @free célèbre aujourd'hui 10 ans de révoluti...,"[{""type"":""video"",""url"":""https://t.co/c1e4svvex...",GroupeIliad,Groupe iliad,https://pbs.twimg.com/profile_images/127365768...,3157961111,NaN,NaN,...,NaN,false,false,false,https://twitter.com/GroupeIliad/status/1480465...,🎂 @free célèbre aujourd'hui 10 ans de révoluti...,False,[],🎂 @free célèbre aujourd'hui 10 ans de révoluti...,0
23,1485331097844129798,2022-01-23 20:18:24 +01:00,Et vous ça se passe comment la fibre ?\n@Sosh_...,"[{""type"":""photo"",""url"":""https://t.co/I41UXroh0...",MitchelMcPat,MitchelMcPat,https://pbs.twimg.com/profile_images/177199285...,906220646492983296,NaN,NaN,...,NaN,false,false,false,https://twitter.com/MitchelMcPat/status/148533...,Et vous ça se passe comment la fibre ? @Sosh_f...,False,[],et vous ça se passe comment la fibre ? @sosh_f...,0
29,1694675653205860558,2023-08-24 13:38:55 +02:00,Dans la lutte contre les installations sauvage...,"[{""type"":""photo"",""url"":""https://t.co/nKKA40veR...",jnbarrot,Jean-Noël Barrot,https://pbs.twimg.com/profile_images/191299208...,865482011523166209,NaN,NaN,...,9127,false,false,false,https://twitter.com/jnbarrot/status/1694675653...,Dans la lutte contre les installations sauvage...,False,[],dans la lutte contre les installations sauvage...,0
30,1695912123824546156,2023-08-27 23:32:13 +02:00,"Hello @free, @Freebox et @orange je crois que ...","[{""type"":""photo"",""url"":""https://t.co/IreryVRyF...",rgaidot,ɹˈe͡ɪɡɪz,https://pbs.twimg.com/profile_images/156717273...,1245801,NaN,NaN,...,778,false,false,false,https://twitter.com/rgaidot/status/16959121238...,"Hello @free, @Freebox et @orange je crois que ...",False,[],"hello @free, @freebox et @orange je crois que ...",0


## 7. Vérification qualitative sur quelques tweets

On regarde quelques exemples pour comparer :

- `text_raw` : texte normalisé de départ ;
- `text_clean_llm` : texte final nettoyé pour le LLM ;
- `hashtags_list` : liste des hashtags détectés.

Objectif : vérifier que les URLs ont disparu, que les `\n` problématiques ont été remplacés, et que les hashtags importants sont toujours présents.


In [16]:
df_clean[["id", "text_raw", "text_clean_llm", "hashtags_list"]].head(10)

,id,text_raw,text_clean_llm,hashtags_list
0,1343458257915031553,"💩 à @free parce-que Débit Très instable, … F...","💩 à @free parce-que débit très instable, … fre...","[#fing, #internet, #Free]"
20,1480465391424053249,🎂 @free célèbre aujourd'hui 10 ans de révoluti...,🎂 @free célèbre aujourd'hui 10 ans de révoluti...,[]
23,1485331097844129798,Et vous ça se passe comment la fibre ? @Sosh_f...,et vous ça se passe comment la fibre ? @sosh_f...,[]
29,1694675653205860558,Dans la lutte contre les installations sauvage...,dans la lutte contre les installations sauvage...,[]
30,1695912123824546156,"Hello @free, @Freebox et @orange je crois que ...","hello @free, @freebox et @orange je crois que ...",[]
31,1709659172361404776,"Bonsoir @WavemakerFR, je suis chez @free et vo...","bonsoir @wavemakerfr, je suis chez @free et vo...",[#Freebox]
32,1711773973401235737,@free @Xavier75 @NThomas82Free En 3 ans l’éq...,@free @xavier75 @nthomas82free en 3 ans l’équi...,[]
36,1759266279620247842,.@free a 25 ans et tient toujours ses promesse...,.@free a 25 ans et tient toujours ses promesse...,[]
37,1762571081402011959,Ça fonctionne super bien la Fibre Free ce soir...,ça fonctionne super bien la fibre free ce soir...,"[#FTTH, #fibre, #Free, #Var]"
42,1811040987365421110,@Xavier75 et tout le réseau @Freebox sachez qu...,@xavier75 et tout le réseau @freebox sachez qu...,[]


## 8. Export des tweets clients nettoyés

On exporte maintenant le DataFrame `df_clean` dans un nouveau fichier CSV
(`free_tweet_export.cleaned.llm.csv`).

Ce fichier contient :

- uniquement des tweets **clients** (tous les comptes Free ont été filtrés),
- une colonne `text_clean_llm` prête pour un LLM,
- une colonne `hashtags_list` avec les hashtags détectés,
- une ligne par tweet unique (déduplication sur `id`).

In [17]:
output_path = Path("../data/free_tweet_export.cleaned.llm.csv")
df_clean.to_csv(output_path, index=False, encoding="utf-8")
output_path

WindowsPath('../data/free_tweet_export.cleaned.llm.csv')

## Interprétation des résultats du nettoyage

Après application du pipeline de nettoyage, on obtient les statistiques suivantes :

- **Tweets au départ (`rows_in`)** : 6 375  
- **Tweets clients après nettoyage (`rows_out` / `tweets_client`)** : 3 044  
- **Tweets des comptes Free filtrés (`reste` / `filtered_handles`)** : 3 331  
- **Retweets réalisés par les comptes Free (`retweets_free`)** : 16  
- **Retweets totaux dans le jeu de données (`retweets_total`)** : 16  
- **Doublons supprimés sur `id` (`dedup_removed`)** : 0  
- **Hashtags perdus pendant le nettoyage (`lost_hashtags_total`)** : 0  

On peut en déduire plusieurs points :

- Le jeu de données initial contenait **6 375 tweets**, dont environ **47,7 % de tweets clients (3 044)** et **52,3 % de tweets provenant des comptes Free (3 331)**.  
  Ces derniers ont été retirés pour ne garder que les interactions des clients.
- Parmi les tweets des comptes Free, **16 étaient des retweets**, soit la totalité des retweets détectés dans le jeu de données (aucun retweet côté clients).
- **Aucun doublon** n’a été détecté sur la colonne `id`, ce qui signifie que chaque tweet client est unique dans le corpus final.
- Le compteur `lost_hashtags_total = 0` montre que **tous les hashtags présents dans les tweets bruts ont été conservés** après nettoyage, ce qui est important pour les futures analyses thématiques ou l’entraînement du LLM.
